We will start creating and understanding how quantum circuits work and are created.

In [1]:
#Import libraries
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib

In [2]:
# Create quantum circuit with 3 qubits and 3 classical bits
#This is our start point

qc = QuantumCircuit(3, 3)
qc.draw()  # returns a drawing of the circuit

q_0: 
     
q_1: 
     
q_2: 
     
c: 3/

Now we will need a way to tell our computer to measure our qubits and record the results. We use the method .measure() in our circuit object to achive this. 

In [3]:
qc = QuantumCircuit(3, 3)
# measure qubits 0, 1 & 2 to classical bits 0, 1 & 2 respectively
qc.measure([0,1,2], [0,1,2])
qc.draw()

┌─┐      
q_0: ┤M├──────
     └╥┘┌─┐   
q_1: ─╫─┤M├───
      ║ └╥┘┌─┐
q_2: ─╫──╫─┤M├
      ║  ║ └╥┘
c: 3/═╩══╩══╩═
      0  1  2

To check the result of this circuit, we'll use a quantum simulator, which is a standard computer calculating what an ideal quantum computer would do.

Simulating a quantum computer is believed to be difficult for classical computers, as the best algorithms we have grow exponentially with the number of qubits. As such, these simulations are only possible for circuits with small numbers of qubits (up to ~30 qubits), or certain types of circuits for which we can use some tricks to speed up the simulation. Nevertheless, simulators are very useful tools for designing smaller quantum circuits.

Let's import Qiskit’s simulator (called Aer), and make a new simulator object.

In [4]:
from qiskit.providers.aer import AerSimulator
sim = AerSimulator()  # make new simulator object

To do the simulation, we can use the simulator’s .run() method. This returns a "job", which contains information about the experiment, such as whether the experiment is running or completed, the backend it ran on, and (importantly for us), the results of the experiment.

To get the results from the job, we use the results method, and the most popular way to view the results is as a dictionary of "counts".

In [5]:
job = sim.run(qc)      # run the experiment
result = job.result()  # get the results
result.get_counts()    # interpret the results as a "counts" dictionary

{'000': 1024}

The keys in counts dictionary are bit-strings, and the values are the number of times that bit-string was measured. Quantum computers can have randomness in their results, so it's common to repeat the circuit a few times. This circuit was repeated 1024 times, which is the default number of times to repeat a circuit in Qiskit.

# Adding 1+1 with a quantum circuit

This is the code that returns the result of adding 1+1 with qiskit:
We will try to break it down. 

We are using the following gates:

    - x gate: change the qubit state
    - cx gate: acts like a XOR gate. It checks the value of the control qubit and gives an x gate on the targeting qubit using the NOT logic. 
    - ccx gate (or Toffoli gate): Same as the cx but checking in two control qubits. It's like an AND gate that creates an X gate if they are both in 1 state. 

In [9]:
#We create a circuit with 4 qubits and 2 bits
test_qc = QuantumCircuit(4, 2)

# First, our circuit should encode an input (here '11')
test_qc.x(0) #.x(n) apply an X gate to the qubit n. 
test_qc.x(1)

# Next, it should carry out the adder circuit we created
test_qc.cx(0,2) #.cx(a,b) CNOT controlled by qubit a and targeting qubit b
test_qc.cx(1,2)
test_qc.ccx(0,1,3) # ccx(a,b,n) AND controlled by a,b qubits and targetint its result in n qubit

# Finally, we will measure the bottom two qubits to extract the output
test_qc.measure(2,0) #measure(a,b) a is the qubit we want to measure in the bit position b
test_qc.measure(3,1)
test_qc.draw()


┌───┐                     
q_0: ┤ X ├──■─────────■────────
     ├───┤  │         │        
q_1: ┤ X ├──┼────■────■────────
     └───┘┌─┴─┐┌─┴─┐  │  ┌─┐   
q_2: ─────┤ X ├┤ X ├──┼──┤M├───
          └───┘└───┘┌─┴─┐└╥┘┌─┐
q_3: ───────────────┤ X ├─╫─┤M├
                    └───┘ ║ └╥┘
c: 2/═════════════════════╩══╩═
                          0  1

In [10]:
job = sim.run(test_qc)  # run the experiment
result = job.result()   # get the results
result.get_counts()     # interpret the results as a “counts” dictionary

{'10': 1024}